In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()
warnings.filterwarnings('ignore')

from llama_index.core import Settings
from llama_index.core.tools.tool_spec.base import BaseToolSpec
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding

import yfinance as yf
from datetime import datetime

import pandas as pd

from typing import Optional, Literal, List, Union, Tuple, Dict

import sys
sys.path.append("../tools")
from forecaster import Forecaster

In [25]:
## Utility functions ##
def rename_columns(ticker: str, df: pd.DataFrame) -> pd.DataFrame:
    """Helper function to post-process dataframe column names"""
    column_dict = {old_name: process_string(ticker = ticker,
                                            string_ = old_name) \
        for old_name in df.columns.values.tolist()}
    df.rename(columns=column_dict, inplace=True)
    return df

def process_string(ticker: str, 
                    string_: str) -> str:
    return f"{ticker}_{'_'.join(string_.lower().split(" "))}"

class FinanceDataTools(BaseToolSpec):
    spec_functions = [
        "get_stock_data",
        "get_min_or_max",
        "get_correlation_between_tickers",
        "get_rolling_average",
        "get_rolling_average_correl",
        "get_longest_uptrend",
        "get_longest_downtrend",
        "cagr",
        "get_statistics",
        "forecast"
    ]
    
    def __init__(self) -> None:
        """Initializes the Yahoo finance tool spec"""
        
    def get_stock_data(self, 
                       ticker: str,
                       period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y") -> pd.DataFrame:
        """Gets the daily historical prices and volume for a ticker across a specified period"""
        return rename_columns(ticker = ticker, df = yf.Ticker(ticker).history(period=period))
    
    def get_min_or_max(self, 
                       ticker: str,
                       field: Literal["Open", 
                                      "High", 
                                      "Low",
                                      "Close",
                                      "Volume",
                                      "Dividends",
                                      "Stock Splits"],
                       period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y",
                       get_max: Optional[bool] = True):
        """Gets min or max value of the field of interest"""
        df = self.get_stock_data(ticker=ticker, period=period)
        field = process_string(ticker=ticker, string_=field)
        if get_max is True:
            value = df[field].max()
        else:
            value = df[field].min()
        return df[df[field] == value]
    
    def get_correlation(self, 
                        ticker: str,
                        period: Optional[
                           Literal["1d",
                                   "5d",
                                   "1mo",
                                   "3mo",
                                   "6mo",
                                   "1y",
                                   "2y",
                                   "5y",
                                   "10y",
                                   "ytd",
                                   "max"]
                           ] = "10y"):
        """Computes correlation between all metrics for a specific ticker"""
        return self.get_stock_data(ticker=ticker, period=period).corr()
    
    def get_correlation_between_tickers(
        self,
        tickers: List[str],
        period: Optional[
                        Literal["1d",
                                "5d",
                                "1mo",
                                "3mo",
                                "6mo",
                                "1y",
                                "2y",
                                "5y",
                                "10y",
                                "ytd",
                                "max"]
                        ] = "10y"
    ):
        """Computes correlation of all metrics for all tickers"""
        df_fin = None
        for ticker in tickers:
            df = self.get_stock_data(ticker=ticker, period=period)
            if df_fin is None:
                df_fin = df
            else:
                df_fin = pd.merge(df_fin, 
                                  df, 
                                  left_index = True,
                                  right_index = True)
        return df_fin.corr()
    
    def get_rolling_average(self, 
                            ticker: str,
                            field: Literal["Open", 
                                      "High", 
                                      "Low",
                                      "Close",
                                      "Volume",
                                      "Dividends",
                                      "Stock Splits"], 
                            n: Optional[int] = 30,
                            period: Optional[
                                        Literal["1d",
                                                "5d",
                                                "1mo",
                                                "3mo",
                                                "6mo",
                                                "1y",
                                                "2y",
                                                "5y",
                                                "10y",
                                                "ytd",
                                                "max"]
                                        ] = "10y"):
        """Computes moving average for field of interest across a specified period"""
        df = self.get_stock_data(ticker=ticker, period=period)
        field = process_string(ticker=ticker, string_=field)
        return pd.DataFrame(df[field].rolling(n).mean())
    
    def get_rolling_average_correl(
        self, 
        tickers: List[str], 
        field: Literal["Open", 
                        "High", 
                        "Low",
                        "Close",
                        "Volume",
                        "Dividends",
                        "Stock Splits"], 
        n: Optional[int] = 30,
        period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y"
    ):
        """Gets correlation of rolling average for a specified field across a list of tickers"""
        df_fin = None
        for ticker in tickers:
            df = self.get_rolling_average(
                ticker = ticker,
                field = field,
                n = n,
                period = period
            )
            if df_fin is None:
                df_fin = df
            else:
                df_fin = pd.merge(df_fin, 
                                  df,
                                  left_index = True,
                                  right_index = True)
        return df_fin.corr()
    
    def get_longest_uptrend(
        self,
        ticker: str
    ):
        """Computes longest stock price uptrend duration for ticker"""
        df = self.get_stock_data(ticker=ticker)
        df['uptrend_days'] = df[f'{ticker}_close'].diff().lt(0).cumsum()
        sizes = df.groupby('uptrend_days')[f'{ticker}_close'].transform('size')
        dates = df[sizes == sizes.max()].index 
        return dates, f"{(dates[-1] - dates[0]).days} days"
    
    def get_longest_downtrend(
        self,
        ticker: str
    ):
        """Computes longest stock price downtrend duration for ticker"""
        df = self.get_stock_data(ticker=ticker)
        df['downtrend_days'] = df[f'{ticker}_close'].diff().gt(0).cumsum()
        sizes=df.groupby('downtrend_days')[f'{ticker}_close'].transform('size')
        dates = df[sizes == sizes.max()].index 
        return dates, f"{(dates[-1] - dates[0]).days} days"
    
    def cagr(self,
             ticker: str,
             period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y"): 
        """Computes compounded annual growth rate of closing prices for specified ticker"""
        df = self.get_stock_data(ticker=ticker, period=period)
        df['year'] = df.index.year
        trimmed = df.iloc[[0, -1]][['year',f'{ticker}_close']]
        n = trimmed['year'].iloc[-1] - trimmed['year'].iloc[0]
        start, end = trimmed[f'{ticker}_close'].iloc[-1], trimmed[f'{ticker}_close'].iloc[0]
        return f"{round(100*((end/start)**(1/n)-1), 1)}%"
    
    def get_statistics(
        self,
        ticker: str,
        period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y",
        grouping: Optional[Literal["year",
                                   "quarter",
                                   "month",
                                   "week",
                                   None]] = None
    ) -> Union[Tuple[float, float], pd.DataFrame]:
        """Gets descriptive statistics of data for grouping of interest"""
        df = self.get_stock_data(ticker=ticker, period=period)
        if grouping is None:
            return df.describe()
        df['year'] = df.index.year
        if grouping == 'year':
            return df.groupby(grouping).mean(), df.groupby(grouping).std()
        if grouping == 'quarter':
            df['quarter'] = df.index.quarter
            return df.groupby(["year", "quarter"]).mean(), df.groupby(["year", "quarter"]).std() 
        if grouping == 'month':
            df['month'] = df.index.month
            return df.groupby(["year", "month"]).mean(), df.groupby(["year", "month"]).std()
    
    def forecast(
        self,
        tickers: List[str],
    ) -> Dict[str, Dict[str, List[float]]]:
        """Develops a quick forecast of the stock prices for the next 3 months
        for a list of tickers at 95% confidence. Use this tool exclusively for
        forecasting future stock prices. 
        
        The models explored here are GARCH, ARCH and Naive forecasting. These models
        first forecast volatility and converts them into forecasted stock prices. The
        tool first uses an "autoARIMA-esque" approach to determine the best model (i.e.
        GARCH/ARCH and their specific hyperparameter configurations), then uses the best
        model from backtesting to forecast forward volatility and subsequently stock 
        prices.
        
        Volatility in this case is defined as logarthmic returns - simply just applying
        the natural logarithm on the ratio of the price of the current period and the
        previous period, where each period refers to the adjusted closing price at the
        monthly interval.s

        Args:
            tickers (List[str]): Tickers of interest
        Returns:
            Dict[str, List[float]]: A dictionary containing the forecasts
            at 95% confidence interval. Example output:
            {
                "Ticker": {
                    "GARCH(1,2)": [..., ..., ...],
                    "GARCH(1,2)-lo-95": [..., ..., ...],
                    "GARCH(1,2)-hi-95": [..., ..., ...],
                }
            }
            
            Whereby "lo-95" refers to the lower boundary  with 95% confidence, 
            and, "hi-95 refers to the high boundary with 95% confidence. The 
            values in the list are forecasted stock prices for the next 3 months.
        """
        forecaster = Forecaster()
        return forecaster(tickers = tickers) 

In [26]:
ft = FinanceDataTools()
ft_tool_list = ft.to_tool_list()

In [27]:
Settings.llm = BedrockConverse(
    model = "anthropic.claude-3-5-sonnet-20240620-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name = "us-east-1",
    max_tokens=4000,
)
Settings.embed_model = BedrockEmbedding(
    model = "amazon.titan-embed-text-v1",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)

In [28]:
from llama_index.core.agent import (
    FunctionCallingAgentWorker,
    StructuredPlannerAgent,
    ReActAgent
)
from IPython.display import display, Markdown

# agent_worker = FunctionCallingAgentWorker.from_tools(
#     tools = ft_tool_list,
#     llm = Settings.llm,
#     verbose = True)
# agent = StructuredPlannerAgent(
#     agent_worker, tools = ft_tool_list, verbose = True
# )
agent = ReActAgent.from_tools(tools=ft_tool_list, verbose = True)

In [29]:
query = "Is there any correlation between Illumina, Apple and NVIDIA closing prices?"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

> Running step 5831032a-6043-4afd-a5cb-d1376dd691fc. Step input: Is there any correlation between Illumina, Apple and NVIDIA closing prices?
Thought: The current language of the user is: English. To answer this question about the correlation between Illumina, Apple, and NVIDIA closing prices, I need to use a tool to gather and analyze the data.
Action: get_correlation_between_tickers
Action Input: {'tickers': ['ILMN', 'AAPL', 'NVDA'], 'period': '10y'}
Observation:                    ILMN_open  ILMN_high  ILMN_low  ILMN_close  ILMN_volume  \
ILMN_open           1.000000   0.999092  0.999196    0.998390    -0.241331   
ILMN_high           0.999092   1.000000  0.998843    0.999191    -0.232780   
ILMN_low            0.999196   0.998843  1.000000    0.999287    -0.247972   
ILMN_close          0.998390   0.999191  0.999287    1.000000    -0.239883   
ILMN_volume        -0.241331  -0.232780 -0.247972   -0.239883     1.000000   
ILMN_dividends           NaN        NaN       NaN         NaN  

<b>Based on the correlation matrix provided, we can analyze the correlation between the closing prices of Illumina (ILMN), Apple (AAPL), and NVIDIA (NVDA) over the past 10 years. Here's what we can conclude:

1. Illumina (ILMN) and Apple (AAPL):
   The correlation between ILMN_close and AAPL_close is 0.133527, which indicates a weak positive correlation. This means that there is a slight tendency for these two stocks to move in the same direction, but the relationship is not strong.

2. Illumina (ILMN) and NVIDIA (NVDA):
   The correlation between ILMN_close and NVDA_close is -0.220059, which suggests a weak negative correlation. This means that there is a slight tendency for these two stocks to move in opposite directions, but again, the relationship is not strong.

3. Apple (AAPL) and NVIDIA (NVDA):
   The correlation between AAPL_close and NVDA_close is 0.780650, which indicates a strong positive correlation. This means that these two stocks tend to move in the same direction quite consistently.

In summary:
- There is a weak positive correlation between Illumina and Apple closing prices.
- There is a weak negative correlation between Illumina and NVIDIA closing prices.
- There is a strong positive correlation between Apple and NVIDIA closing prices.

It's important to note that while Apple and NVIDIA show a strong correlation, Illumina's price movements are not strongly correlated with either of the other two stocks. This suggests that Illumina's stock price may be influenced by different factors compared to Apple and NVIDIA, which seem to move more in tandem with each other.</b>

In [30]:
query = "Forecast the stock prices of Apple, NVIDIA and Illumina. Which stock should I invest in and which stock should I short?"
response = agent.chat(query)

display(Markdown(f"<b>{response}</b>"))

> Running step eb94e80d-a52b-4cac-b21e-4e10186af3a6. Step input: Forecast the stock prices of Apple, NVIDIA and Illumina. Which stock should I invest in and which stock should I short?


[*********************100%%**********************]  3 of 3 completed

Thought: I need to use the forecast tool to get predictions for these three stocks.
Action: forecast
Action Input: {'tickers': ['AAPL', 'NVDA', 'ILMN']}



Task exception was never retrieved
future: <Task finished name='Task-150' coro=<Dispatcher.span.<locals>.async_wrapper() done, defined at /App/tlim2/Anaconda3/envs/llamaindex/lib/python3.12/site-packages/llama_index/core/instrumentation/dispatcher.py:244> exception=ValidationException('An error occurred (ValidationException) when calling the Converse operation: Expected toolResult blocks at messages.0.content for the following Ids: tooluse_uLD0SDkKQsmfpTmpTUrCtQ')>
Traceback (most recent call last):
  File "/App/tlim2/Anaconda3/envs/llamaindex/lib/python3.12/asyncio/tasks.py", line 316, in __step_run_and_handle_result
    result = coro.throw(exc)
             ^^^^^^^^^^^^^^^
  File "/App/tlim2/Anaconda3/envs/llamaindex/lib/python3.12/site-packages/llama_index/core/instrumentation/dispatcher.py", line 255, in async_wrapper
    result = await func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/App/tlim2/Anaconda3/envs/llamaindex/lib/python3.12/site-packages/llama_ind

Observation: {'AAPL':    GARCH(2,2)  GARCH(2,2)_95%_Confidence_Interval_Low  \
0  265.186410                              272.539987   
1  245.254407                              244.123401   
2  229.216829                              220.944956   

   GARCH(2,2)_80%_Confidence_Interval_Low  \
0                              265.532041   
1                              240.493105   
2                              220.093712   

   GARCH(2,2)_80%_Confidence_Interval_High  \
0                               275.426188   
1                               260.106126   
2                               248.258977   

   GARCH(2,2)_95%_Confidence_Interval_High  
0                               314.015906  
1                               299.849642  
2                               289.393162  , 'NVDA':    GARCH(1,1)  GARCH(1,1)_95%_Confidence_Interval_Low  \
0  130.912338                              134.367715   
1  120.047121                              118.465413   
2  111.558932          

<b>Based on the forecasts provided, here's an analysis of each stock and recommendations:

1. Apple (AAPL):
   The forecast shows a downward trend for Apple's stock price over the next three months. The GARCH(2,2) model predicts:
   - Month 1: $265.19
   - Month 2: $245.25
   - Month 3: $229.22
   This represents a significant decline from the current price.

2. NVIDIA (NVDA):
   NVIDIA's forecast also shows a downward trend, but less steep than Apple's:
   - Month 1: $130.91
   - Month 2: $120.05
   - Month 3: $111.56

3. Illumina (ILMN):
   Illumina's forecast follows a similar downward pattern:
   - Month 1: $148.87
   - Month 2: $138.78
   - Month 3: $130.62

Investment Advice:

1. Stock to Invest in:
   Given that all three stocks show downward trends, it's challenging to recommend a clear "buy" option. However, if you must choose one to invest in, NVIDIA (NVDA) might be the best option among the three. It shows the least steep decline and has historically been a strong performer in the tech sector. The semiconductor industry, which NVIDIA leads, has long-term growth potential due to AI and other emerging technologies.

2. Stock to Short:
   Apple (AAPL) shows the steepest predicted decline among the three stocks. If you're considering shorting a stock, Apple might be the best candidate based on this forecast. The model predicts a more significant drop compared to NVIDIA and Illumina.

Important Caveats:
1. These forecasts are predictions and not guarantees. The stock market is influenced by many factors that models can't always account for.
2. The confidence intervals for all stocks are quite wide, indicating significant uncertainty in these predictions.
3. Short-term trading based on such forecasts is risky. These predictions are for a 3-month period, which is relatively short in investment terms.
4. Always consider your risk tolerance and conduct thorough research before making investment decisions.
5. It's advisable to diversify your investments rather than focusing on a single stock or strategy.

Remember, this advice is based solely on the provided forecasts and should not be considered as professional financial advice. It's always recommended to consult with a financial advisor before making significant investment decisions, especially when considering high-risk strategies like shorting stocks.</b>

## Export

In [1]:
#%%
from llama_index.core.tools.tool_spec.base import BaseToolSpec
import yfinance as yf
import pandas as pd
from typing import Optional, Literal, List, Union, Tuple, Dict

from forecaster import Forecaster

import os
import sys
__curdir__ = os.getcwd()

if ("tools" in __curdir__) or \
    ("agents" in __curdir__) or \
    ("tasks" in __curdir__):
    sys.path.append(os.path.join(
        __curdir__,
        "../src"
    ))
else:
    sys.path.append("./src")
from utils import rename_columns, process_string

import warnings
warnings.filterwarnings('ignore')


#%%
class DataAnalysisTools(BaseToolSpec):
    """These tools are intended for general data analysis by agents. They allow for very general
    questions and specific questions by users."""
    
    spec_functions = [
        "get_stock_data",
        "get_min_or_max",
        "get_correlation_between_tickers",
        "get_rolling_average",
        "get_rolling_average_correl",
        "get_longest_uptrend",
        "get_longest_downtrend",
        "cagr",
        "get_statistics",
        "forecast"
    ]
    
    def __init__(self) -> None:
        """Initializes the Yahoo finance tool spec"""
        
    def get_stock_data(
        self, 
        ticker: str,
        period: Optional[
            Literal["1d",
                    "5d",
                    "1mo",
                    "3mo",
                    "6mo",
                    "1y",
                    "2y",
                    "5y",
                    "10y",
                    "ytd",
                    "max"]
            ] = "10y") -> pd.DataFrame:
        """Gets the daily historical prices and volume for a ticker across a specified 
        period"""
        return rename_columns(ticker = ticker, df = yf.Ticker(ticker).history(
            period=period
        ))
    
    def get_min_or_max(
        self, 
        ticker: str,
        field: Literal["Open", 
                        "High", 
                        "Low",
                        "Close",
                        "Volume",
                        "Dividends",
                        "Stock Splits"],
        period: Optional[
            Literal["1d",
                    "5d",
                    "1mo",
                    "3mo",
                    "6mo",
                    "1y",
                    "2y",
                    "5y",
                    "10y",
                    "ytd",
                    "max"]
            ] = "10y",
        get_max: Optional[bool] = True):
        """Gets min or max value of the field of interest"""
        df = self.get_stock_data(ticker=ticker, period=period)
        field = process_string(ticker=ticker, string_=field)
        if get_max is True:
            value = df[field].max()
        else:
            value = df[field].min()
        return df[df[field] == value]
    
    def get_correlation(
        self, 
        ticker: str,
        period: Optional[
            Literal["1d",
                    "5d",
                    "1mo",
                    "3mo",
                    "6mo",
                    "1y",
                    "2y",
                    "5y",
                    "10y",
                    "ytd",
                    "max"]
            ] = "10y"):
        """Computes correlation between all metrics for a specific ticker"""
        return self.get_stock_data(ticker=ticker, period=period).corr()
    
    def get_correlation_between_tickers(
        self,
        tickers: List[str],
        period: Optional[
                        Literal["1d",
                                "5d",
                                "1mo",
                                "3mo",
                                "6mo",
                                "1y",
                                "2y",
                                "5y",
                                "10y",
                                "ytd",
                                "max"]
                        ] = "10y"
    ):
        """Computes correlation of all metrics for all tickers"""
        df_fin = None
        for ticker in tickers:
            df = self.get_stock_data(ticker=ticker, period=period)
            if df_fin is None:
                df_fin = df
            else:
                df_fin = pd.merge(
                    df_fin, 
                    df, 
                    left_index = True,
                    right_index = True)
        return df_fin.corr()
    
    def get_rolling_average(
        self, 
        ticker: str,
        field: Literal["Open", 
                    "High", 
                    "Low",
                    "Close",
                    "Volume",
                    "Dividends",
                    "Stock Splits"], 
        n: Optional[int] = 30,
        period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y"):
        """Computes moving average for field of interest across a specified period"""
        df = self.get_stock_data(ticker=ticker, period=period)
        field = process_string(ticker=ticker, string_=field)
        return pd.DataFrame(df[field].rolling(n).mean())
    
    def get_rolling_average_correl(
        self, 
        tickers: List[str], 
        field: Literal["Open", 
                        "High", 
                        "Low",
                        "Close",
                        "Volume",
                        "Dividends",
                        "Stock Splits"], 
        n: Optional[int] = 30,
        period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y"
    ):
        """Gets correlation of rolling average for a specified field across a list of 
        tickers"""
        
        df_fin = None
        for ticker in tickers:
            df = self.get_rolling_average(
                ticker = ticker,
                field = field,
                n = n,
                period = period
            )
            if df_fin is None:
                df_fin = df
            else:
                df_fin = pd.merge(
                    df_fin, 
                    df,
                    left_index = True,
                    right_index = True)
        return df_fin.corr()
    
    def get_longest_uptrend(
        self,
        ticker: str
    ):
        """Computes longest stock price uptrend duration for ticker"""
        df = self.get_stock_data(ticker=ticker)
        df['uptrend_days'] = df[f'{ticker}_close'].diff().lt(0).cumsum()
        sizes = df.groupby('uptrend_days')[f'{ticker}_close'].transform('size')
        dates = df[sizes == sizes.max()].index 
        return dates, f"{(dates[-1] - dates[0]).days} days"
    
    def get_longest_downtrend(
        self,
        ticker: str
    ):
        """Computes longest stock price downtrend duration for ticker"""
        df = self.get_stock_data(ticker=ticker)
        df['downtrend_days'] = df[f'{ticker}_close'].diff().gt(0).cumsum()
        sizes=df.groupby('downtrend_days')[f'{ticker}_close'].transform('size')
        dates = df[sizes == sizes.max()].index 
        return dates, f"{(dates[-1] - dates[0]).days} days"
    
    def cagr(
        self,
        ticker: str,
        period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y"): 
        """Computes compounded annual growth rate of closing prices for specified 
        ticker"""
        df = self.get_stock_data(ticker=ticker, period=period)
        df['year'] = df.index.year
        trimmed = df.iloc[[0, -1]][['year',f'{ticker}_close']]
        n = trimmed['year'].iloc[-1] - trimmed['year'].iloc[0]
        start, end = trimmed[f'{ticker}_close'].iloc[-1], trimmed[f'{ticker}_close'].iloc[0]
        return f"{round(100*((end/start)**(1/n)-1), 1)}%"
    
    def get_statistics(
        self,
        ticker: str,
        period: Optional[
                    Literal["1d",
                            "5d",
                            "1mo",
                            "3mo",
                            "6mo",
                            "1y",
                            "2y",
                            "5y",
                            "10y",
                            "ytd",
                            "max"]
                    ] = "10y",
        grouping: Optional[
            Literal["year",
                    "quarter",
                    "month",
                    "week",
                    None]] = None
    ) -> Union[Tuple[float, float], pd.DataFrame]:
        """Gets descriptive statistics of data for grouping of interest"""
        df = self.get_stock_data(ticker=ticker, period=period)
        if grouping is None:
            return df.describe()
        df['year'] = df.index.year
        if grouping == 'year':
            return df.groupby(grouping).mean(), df.groupby(grouping).std()
        if grouping == 'quarter':
            df['quarter'] = df.index.quarter
            return df.groupby(["year", "quarter"]).mean(), df.groupby(["year", "quarter"]).std() 
        if grouping == 'month':
            df['month'] = df.index.month
            return df.groupby(["year", "month"]).mean(), df.groupby(["year", "month"]).std()
    
    def forecast(
        self,
        tickers: List[str],
    ) -> Dict[str, Dict[str, List[float]]]:
        """Develops a quick forecast of the stock prices for the next 3 months
        for a list of tickers at 95% confidence. Use this tool exclusively for
        forecasting future stock prices. 
        
        The models explored here are GARCH, ARCH and Naive forecasting. These models
        first forecast volatility and converts them into forecasted stock prices. The
        tool first uses an "autoARIMA-esque" approach to determine the best model (i.e.
        GARCH/ARCH and their specific hyperparameter configurations), then uses the best
        model from backtesting to forecast forward volatility and subsequently stock 
        prices.
        
        Volatility in this case is defined as logarthmic returns - simply just applying
        the natural logarithm on the ratio of the price of the current period and the
        previous period, where each period refers to the adjusted closing price at the
        monthly interval.s

        Args:
            tickers (List[str]): Tickers of interest
        Returns:
            Dict[str, List[float]]: A dictionary containing the forecasts
            at 95% confidence interval. Example output:
            {
                "Ticker": {
                    "GARCH(1,2)": [..., ..., ...],
                    "GARCH(1,2)-lo-95": [..., ..., ...],
                    "GARCH(1,2)-hi-95": [..., ..., ...],
                }
            }
            
            Whereby "lo-95" refers to the lower boundary  with 95% confidence, 
            and, "hi-95 refers to the high boundary with 95% confidence. The 
            values in the list are forecasted stock prices for the next 3 months.
        """
        forecaster = Forecaster()
        return forecaster(tickers = tickers)

def get_da_tools():
    da = DataAnalysisTools()
    return da.to_tool_list()  


Overwriting data_analysis_tools.py
